In [6]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 12.9 MB/s eta 0:00:00


# Service 1

In [7]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
import optuna

# Load and preprocess data
df = pd.read_csv("Service1.csv")
features = ['latency_ms', 'cpu_allocated', 'memory_allocated', 'cpu_usage_pct', 'memory_usage_pct']
scaler = MinMaxScaler()
df[features] = scaler.fit_transform(df[features])
X = df[features].values
y = df[['cpu_usage_pct', 'memory_usage_pct']].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.FloatTensor(y_test)

# Define PPO Actor and Critic Networks
class Actor(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.model(x)

class Critic(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )
    def forward(self, x):
        return self.model(x)

# Helper to inverse scale output
def inverse_targets(y_pred):
    dummy = np.zeros((y_pred.shape[0], 5))
    dummy[:, -2:] = y_pred
    return scaler.inverse_transform(dummy)[:, -2:]

# Evaluation function
def evaluate(actor):
    actor.eval()
    with torch.no_grad():
        y_pred = actor(X_test_tensor).numpy()
    y_pred_inv = inverse_targets(y_pred)
    y_test_inv = inverse_targets(y_test)
    mae_cpu = mean_absolute_error(y_test_inv[:, 0], y_pred_inv[:, 0])
    mae_mem = mean_absolute_error(y_test_inv[:, 1], y_pred_inv[:, 1])
    return mae_cpu + mae_mem

# Optuna objective function
def objective(trial):
    hidden_dim = trial.suggest_int("hidden_dim", 64, 256)
    actor_lr = trial.suggest_float("actor_lr", 1e-4, 1e-2, log=True)
    critic_lr = trial.suggest_float("critic_lr", 1e-4, 1e-2, log=True)
    gamma = trial.suggest_float("gamma", 0.9, 0.99)
    eps_clip = trial.suggest_float("eps_clip", 0.1, 0.3)
    epochs = trial.suggest_int("epochs", 30, 100)

    actor = Actor(input_dim=5, output_dim=2, hidden_dim=hidden_dim)
    critic = Critic(input_dim=5, hidden_dim=hidden_dim)
    optimizer_actor = optim.Adam(actor.parameters(), lr=actor_lr)
    optimizer_critic = optim.Adam(critic.parameters(), lr=critic_lr)

    for _ in range(epochs):
        old_actions = actor(X_train_tensor).detach()
        values = critic(X_train_tensor).squeeze()
        rewards = -((old_actions - y_train_tensor) ** 2).mean(dim=1)
        advantages = rewards - values.detach()

        # Critic update
        critic_loss = nn.MSELoss()(values, rewards)
        optimizer_critic.zero_grad()
        critic_loss.backward()
        optimizer_critic.step()

        # Actor update
        new_actions = actor(X_train_tensor)
        ratios = ((new_actions - old_actions) ** 2).sum(dim=1)
        clipped = torch.clamp(ratios, 1 - eps_clip, 1 + eps_clip)
        actor_loss = -torch.min(ratios * advantages, clipped * advantages).mean()
        optimizer_actor.zero_grad()
        actor_loss.backward()
        optimizer_actor.step()

    return evaluate(actor)

# Start tuning
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30)

print("Best Hyperparameters:", study.best_params)
print("Best MAE (CPU + MEM):", study.best_value)


[I 2025-05-05 10:03:15,519] A new study created in memory with name: no-name-3b37dfba-57f8-421c-ba3c-2b9265d3645c
[I 2025-05-05 10:03:20,003] Trial 0 finished with value: 68.60466259322703 and parameters: {'hidden_dim': 85, 'actor_lr': 0.0026434392234761946, 'critic_lr': 0.0020038716813092034, 'gamma': 0.9863038435039437, 'eps_clip': 0.10111554887593202, 'epochs': 44}. Best is trial 0 with value: 68.60466259322703.
[I 2025-05-05 10:03:27,718] Trial 1 finished with value: 70.29046618499308 and parameters: {'hidden_dim': 188, 'actor_lr': 0.0003103820509045979, 'critic_lr': 0.0001750451971262604, 'gamma': 0.9611219622337349, 'eps_clip': 0.15307189875256436, 'epochs': 42}. Best is trial 0 with value: 68.60466259322703.
[I 2025-05-05 10:03:34,046] Trial 2 finished with value: 68.57703648499101 and parameters: {'hidden_dim': 168, 'actor_lr': 0.0011453183147069029, 'critic_lr': 0.0026569658578447, 'gamma': 0.9497409833858836, 'eps_clip': 0.24725819304374846, 'epochs': 52}. Best is trial 2 wit

Best Hyperparameters: {'hidden_dim': 156, 'actor_lr': 0.00045986299134188793, 'critic_lr': 0.003411812340891027, 'gamma': 0.954757535643432, 'eps_clip': 0.1371682064390806, 'epochs': 71}
Best MAE (CPU + MEM): 66.57720566642939
